In [ ]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog
pd.set_option("display.max_columns", None)
pd.set_option("mode.chained_assignment", None)

import warnings
warnings.filterwarnings("ignore")
from thefuzz import fuzz
from thefuzz import process

In [ ]:
class SizeDataTransformer:
    
    def __init__(self, path):
        self.path = path
        
    def read_data(self):
        if self.path.split(".")[-1] == "xlsx":
            df = pd.read_excel(self.path, None)
            catalog = {}
            for i, j in enumerate(list(df.keys())):
                catalog.setdefault(i + 1, j)
                print(i + 1, j)
            answer = int(input("Please select the sheet"))
            df = df[catalog[answer]]
        else:
            df = pd.read_csv(self.path)
        return df
    
    @staticmethod
    def tire_type(df, category, channel=None, years=True):
        if years == True:
            df = df[(df["KA分類"] == category) & (df["大通路分類"] == channel) & (df["年月"].astype(str).str.contains("2022|2023"))]
        else:
            df = df[(df["KA分類"] == category) & (df["大通路分類"] == channel)]
        df["整理後的日期"] = pd.to_datetime(df["年月"], format="%Y%m")
        return df
        
        
        
    
    

In [ ]:
root = tk.Tk()
root.withdraw()
file_path = filedialog.askopenfilename()

In [ ]:
df = SizeDataTransformer(file_path)

In [ ]:
df = df.read_data()

In [ ]:
result = SizeDataTransformer.tire_type(df, "TBR(G)", "REP", False)

In [ ]:
result["條數"] = result["條數"].fillna(0)

In [ ]:
result.info()

In [ ]:
result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\2021-2022_Size Data_All_TBR_rawling_data.xlsx", index=False)


In [ ]:
df = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data.xlsx", None)
df.keys()

In [ ]:
df = df["工作表1"]

In [ ]:
df = df[(df["大通路分類"] == "REP") & (df["國內市場財別"] == "TBR(G)")]
df.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data_TBR版本.xlsx", index=False)

In [ ]:
df = pd.read_clipboard()

In [ ]:
result = df[(df["Size Designation"].str.startswith("P") == False) & (df["PrCode"].str.contains("TBR|LSR"))]

In [ ]:
bpcom_dict = dict(zip(result["PrCode"], result["Product Name"]))

In [ ]:
new_rule = pd.read_clipboard()

In [ ]:
new_rule_dict = dict(zip(new_rule["Product name"], new_rule["Category"]))

In [ ]:
product_list = new_rule["Product name"].values.tolist()

In [ ]:
isom = pd.read_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data_TBR版本.xlsx")
isom

In [ ]:
isom["Product Name"] = isom["商品代號"].map(bpcom_dict)

In [ ]:
isom["Product Name"] = isom["Product Name"].fillna("na")

In [ ]:
final_result = pd.concat([isom, pd.DataFrame(isom["Product Name"].map(lambda x: process.extractOne(x, product_list)).tolist(), columns=["匹配商品名稱", "Confidence"])], axis=1)

In [ ]:
final_result["New Category"] = final_result["匹配商品名稱"].map(new_rule_dict)

In [ ]:
final_result.to_excel(r"D:\kc.hsu\OneDrive - Bridgestone\數據\2022ACT\202301 ISOM Size data_TBR版本.xlsx", index=False)